In [5]:
import numpy as np

In [8]:
class GeneticAlgorithm:

    def __init__(
            self,
            tasks_count: int,
            tasks: np.ndarray,
            tasks_time: np.ndarray,
            devs_count: int,
            coefficients: np.ndarray
    ):
        self.tasks_count = tasks_count
        self.tasks = tasks
        self.tasks_time = tasks_time
        self.devs_count = devs_count
        self.coefficients = coefficients

        self.rng = np.random.default_rng()
        self.population_size = 1500
        self.max_generations = 250

        self.population = self.rng.integers(low=0, high=self.devs_count, size=(self.population_size, self.tasks_count))

    def fitness(self, individual):
        total_time = np.zeros(self.devs_count)
        for i in range(self.tasks_count):
            total_time[individual[i]] += self.tasks_time[i] * self.coefficients[individual[i], self.tasks[i] - 1]
        return np.max(total_time)

    def selection(self):
        # турирная селекция
        fitness_values = np.array([self.fitness(ind) for ind in self.population])
        selected_indices = np.argsort(fitness_values)[:self.population_size // 2 * 2]
        return self.population[selected_indices]

    def crossover(self, parent1, parent2):
        # скрещивание с помощью одноточечного кроссовера
        crossover_point = self.rng.integers(low=1, high=self.tasks_count)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2

    def mutation(self, individual):
        # мутация с некоторой вероятностью (0.01, условие ниже)
        mutated_gene = self.rng.integers(low=0, high=self.devs_count)
        mutation_point = self.rng.integers(low=0, high=self.tasks_count)
        individual[mutation_point] = mutated_gene
        return individual

    def evolve(self):
        for generation in range(self.max_generations):
            selected_population = self.selection()

            new_population = []
            for i in range(0, len(selected_population), 2):
                parent1, parent2 = selected_population[i], selected_population[i + 1]
                crossover_point = self.rng.integers(low=1, high=self.tasks_count, size=1)[0]

                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

                child1 = self.mutation(child1) if self.rng.random() < 0.01 else child1
                child2 = self.mutation(child2) if self.rng.random() < 0.01 else child2

                new_population.extend([child1, child2])

            top_percent = int(0.5 * self.population_size)  # выбор 50 % детей
            self.population[:top_percent] = self.selection()[:top_percent]
            self.population[top_percent:] = np.array(new_population)[:self.population_size - top_percent]

            best_fitness = self.fitness(self.population[np.argmin([self.fitness(ind) for ind in self.population])])

            print(f"Поколение {generation + 1}: {best_fitness}")

        best_fitness = self.population[np.argmin([self.fitness(ind) for ind in self.population])]
        return best_fitness

In [9]:
# Чтение входных данных из файла
with open('input.txt') as f:
    tasks_count = int(f.readline())
    tasks = np.array(list(map(int, f.readline().split())))
    tasks_time = np.array(list(map(float, f.readline().split())))
    devs_count = int(f.readline())
    coefficients = np.array([list(map(float, f.readline().split())) for i in range(devs_count)])

ga = GeneticAlgorithm(tasks_count, tasks, tasks_time, devs_count, coefficients)

best_solution = ga.evolve()

with open('output.txt', 'w') as file:
    file.write(" ".join(map(lambda x: str(x + 1), best_solution)))

best_developer_times = np.zeros(ga.devs_count)
for i in range(ga.tasks_count):
    best_developer_times[best_solution[i]] += ga.tasks_time[i] * ga.coefficients[best_solution[i], ga.tasks[i] - 1]

Tmax = np.max(best_developer_times)

score = ((10 ** 6) / Tmax) / 10 ** 4
print("Score:", score)

Поколение 1: 659.1100000000002
Поколение 2: 659.1100000000002
Поколение 3: 647.5200000000001
Поколение 4: 647.5200000000001
Поколение 5: 647.5200000000001
Поколение 6: 638.4600000000002
Поколение 7: 638.4600000000002
Поколение 8: 632.5049999999997
Поколение 9: 632.5049999999997
Поколение 10: 632.5049999999997
Поколение 11: 632.5049999999997
Поколение 12: 631.4050000000003
Поколение 13: 631.4050000000003
Поколение 14: 631.4050000000003
Поколение 15: 630.685
Поколение 16: 630.685
Поколение 17: 629.085
Поколение 18: 626.6750000000001
Поколение 19: 626.6750000000001
Поколение 20: 626.6750000000001
Поколение 21: 626.6750000000001
Поколение 22: 626.5050000000001
Поколение 23: 626.5050000000001
Поколение 24: 626.5050000000001
Поколение 25: 626.5050000000001
Поколение 26: 626.5050000000001
Поколение 27: 626.165
Поколение 28: 626.165
Поколение 29: 626.165
Поколение 30: 626.165
Поколение 31: 626.165
Поколение 32: 624.6699999999998
Поколение 33: 624.6699999999998
Поколение 34: 624.6699999999998
П